In [121]:
import psycopg2 as ps
import pandas as pd
import Preprocessing
import time
import numpy as np

In [2]:
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()
q = '''SELECT * FROM TWEETS
        WHERE TWEET_CREATED >= '2018-01-01' 
        AND TWEET_CREATED < '2021-01-01'; '''
cursor.execute(q)
results = cursor.fetchall()
colnames = [desc[0] 
        for desc in cursor.description]
cursor.close()
conn.close()

df = pd.DataFrame(results, columns=colnames)

In [4]:
start = time.time()
df_clean = Preprocessing.clean_text(df, 'tweet_text')
end = time.time()
print(str(end-start), 'ms')

11.453316688537598 ms


In [5]:
df_clean.clean_text = df_clean.clean.copy()
df_clean = df_clean.drop('clean', axis=1)

In [6]:
# shift column 'Name' to first position
clean_text = df_clean.pop('clean_text') 
# insert column using insert(position,column_name,
# first_column) function
df_clean.insert(6, 'clean_text', clean_text)

In [99]:
short_df = df_clean[['tweet_id', 'clean_text', 'isincident', 'isaccident', 'isobstacle', 'isdanger', 'incident_desc']].copy()

In [3]:
short_df

,Unnamed: 0,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc
0,0,1013816364220518402,precaucion calle 50 anegada via,NaN,NaN,NaN,NaN,NaN
1,1,951132392906870784,recorrido mañanitas,NaN,NaN,NaN,NaN,NaN
2,2,1010612149826342914,inundada calle 70 san fransisco,NaN,NaN,NaN,NaN,NaN
3,3,1013816934142562304,inundada calle 74 san francisco precaucion ruta,NaN,NaN,NaN,NaN,NaN
4,4,1022561878969671680,inundada avenida frente mercado marisco,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
87101,87101,1344759940305612820,5km h desplazan autos via transistmica altura ...,NaN,NaN,NaN,NaN,NaN
87102,87102,1344773829290287105,recibe resumen noticias capsulas video canal y...,NaN,NaN,NaN,NaN,NaN
87103,87103,1344780610594856967,celebremos viven casa forma virtual expresemos...,NaN,NaN,NaN,NaN,NaN
87104,87104,1344810167276363781,registra accidente transito via centenario alt...,True,True,NaN,NaN,accidente


In [301]:
short_df = pd.read_csv('tweets-2018-2020-clean.csv')
short_df.drop('Unnamed: 0', axis=1, inplace=True)
short_df = short_df.where(pd.notnull(short_df), None)

In [302]:
short_df = short_df.where(pd.notnull(short_df), None)

In [450]:
len(short_df[short_df.isincident.isnull()])

60784

In [12]:
pd.set_option('display.max_columns', 500)
pd.set_option("max_colwidth", 200)
pd.set_option('display.max_rows', 500)

# Accidentes

In [448]:
extract = short_df.clean_text.str.contains('atropell[ a-z]*') & short_df.isaccident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc
112,948305062664835072,atropello via interamericana sajalices chame victima permanece pavimento,None,None,None,None,None
121,948315622777278466,registra atropello via interamericana altura sajaciles precaucion trafico afectado ambas direcciones,None,None,None,None,None
190,948948750323666952,persona atropella cinta costera frente hospital niño,None,None,None,None,None
191,948949356157263873,hora camion atropella hombre cinta costera frente hospital niño,None,None,None,None,None
621,951296667919233025,fallece menor años edad atropello via interamericana espino santiago veraguas informes camion transitaba via infante descuido padres salio residencia trato cruzar via impactado vehiculo,None,None,None,None,None
...,...,...,...,...,...,...,...
86430,1341030496353050624,registra atropello avenida pulida motorizado afectado precaucion ruta,None,None,None,None,None
86432,1341033963406671873,reportan victima fatal atropello autopista panama colon altura 4 altos mantiene trafico paralizado foto,None,None,None,None,None
86440,1341057759572332549,trafico paralizado autopista panama colon altura 4 altos provincia colon victima fatal atropello,None,None,None,None,None
86452,1341078646556135424,trafico paralizado autopista panama colon altura 4 altos provincia colon victima fatal atropello,None,None,None,None,None


In [442]:
short_df.loc[extract, 'isincident'] = True
short_df.loc[extract, 'isaccident'] = True
# short_df.loc[extract, 'isobstacle'] = True
short_df.loc[extract, 'incident_desc'] = 'accidente'

In [336]:
short_df.loc[753, 'isdanger'] = False

In [374]:
extract = short_df.clean_text.str.contains('regist.* [0-9]+[ a-z]*accidente[s]?') & short_df.isaccident==True
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc


In [189]:
# short_df.loc[extract, 'isincident'] = None
short_df.loc[extract, 'isaccident'] = False
# short_df.loc[extract, 'isobstacle'] = False
# short_df.loc[extract, 'isdanger'] = False
# short_df.loc[extract, 'incident_desc'] = None

In [372]:
short_df[extract].to_csv('colision-transito.csv', encoding='utf-8')

In [381]:
revision = pd.read_csv('colision-transito.csv')
revision = revision.where(pd.notnull(revision), None)

In [399]:
len(revision)

394

In [400]:
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'isaccident'] = True
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==0].tweet_id), 'isaccident'] = False
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'isincident'] = True
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'incident_desc'] = 'colision'
short_df.loc[short_df.tweet_id.isin(revision.tweet_id), :]

# Peligros

In [289]:
extract = short_df.clean_text.str.contains('inundacion(es)?') & short_df.isincident.isnull()
short_df[extract]

C:\Users\lucia\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc
392,950038692466393089,ciudad sol margarita provincia colon mantiene cerrada calle carros atascados inundaciones,None,None,None,None,None
393,950040601776525314,mantiene cerrada calle ciudad sol margarita provincia colon carros atascados inundaciones via,None,None,None,None,None
1063,953682505864970241,inundacion avenida martires altura chorrillo direccion panama centro via,None,None,None,None,None
3710,967852415713267712,carros afectados inundaciones via tumba muerto altura cochez direccion san miguelito evitar ruta,None,None,None,None,None
3711,967853147850002432,carros cae cuneta medio inundaciones via tumba muerto altura cochez direccion san miguelito via,None,None,None,None,None
3712,967853235548672000,carros afectados inundaciones via tumba muerto altura cochez direccion san miguelito,None,None,None,None,None
3716,967858024617594883,vistas carros afectados inundaciones via tumba muerto altura cochez direccion san miguelito evitar ruta,None,None,None,None,None
7187,983441873615409153,registran inundaciones avenida balboa altura marbella recuerde enviar reporte utilizando etiqueta,None,None,None,None,None
7190,983444404559073280,inundaciones calle 50 altura credicorp bank recuerde manejar cuidado,None,None,None,None,None
7194,983449339799490562,video inundaciones avenida balboa altura marbella,None,None,None,None,None


In [288]:
short_df.loc[extract, 'isincident'] = True
short_df.loc[extract, 'isdanger'] = True
short_df.loc[extract, 'isobstacle'] = True
short_df.loc[extract, 'incident_desc'] = 'tranque, lluvias'

In [449]:
short_df.to_csv('tweets-2018-2020-clean.csv', encoding='utf-8')

In [9]:
# for i in range(2014, 2021):
#     df_clean[(df_clean.tweet_created>='%i-01-01'%i) & (df_clean.tweet_created<'%i-01-01'%(i+1))].to_csv('tweets_%i.csv'%(i), encoding='utf-8')
df_clean.to_csv('tweets-2018-2020.csv', encoding='utf-8')